# tools for pre-processing images

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
__author__ = 'Pascal Louis <plouis35@gmail.com>'
__version__ = '0.1 alpha'
__keywords__ = ['astronomy', 'astropy', 'astroalign', 'ccdproc', 'matplotlib']


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.rc = {}
plt.style.use(['dark_background'])
plt.rcParams.update({'figure.max_open_warning': 0})      

def show_image(img):
    #img = lights_c[1]
    #p2, p98 = np.percentile(img, (2, 98))
    #l_rescale = sk.exposure.rescale_intensity(img, in_range=(p2, p98))
    #plt.imshow(l_rescale / np.max(l_rescale), cmap="gray")
    colormap = plt.cm.inferno
    cuts = (5, 98.0)
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    plt.tight_layout()
    im = ax.imshow(img, origin='lower', aspect='auto', cmap=colormap)
    cb = plt.colorbar(im)
    im.set_clim(np.percentile(img, cuts))
    plt.show()


In [ ]:
from logger_utils import logger, handler
handler.show_logs()
logger.setLevel('INFO')
handler.clear_logs()


In [ ]:
CAPTURE_DIR='../../../CAPTURES/NY_Vir/'
BIAS_FILES = "offset-15c-Bin2-*.fits"
DARK_FILES = "darkBin2X-35C-30s-*.fits"
FLAT_FILES = "Flat_v-*.fits"
CALIB_FILES = ""
SCIENCE_FILES = "NY VIR2*.fits"
TRIM_REGION = None
EXPOSURE_KEY = 'EXPOSURE'

In [ ]:
CAPTURE_DIR='../../../CAPTURES/2023_RENNES_L3_TP/images_spectro/images_tp_spectro/'
BIAS_FILES = "offset-*.fit"
DARK_FILES = "noir-600-*.fit"
FLAT_FILES = "flat-*.fit"
CALIB_FILES = "neon-*.fit"
SCIENCE_FILES = "agdra-*.fit"
TRIM_REGION = '600, 600, 2700, 1400'              # x1, y1, x2, y2 
#TRIM_REGION = None
EXPOSURE_KEY = 'EXPTIME'

In [ ]:
CAPTURE_DIR='../../../CAPTURES/20231008_Void/'
BIAS_FILES = "Bias-0s-*.fit"
DARK_FILES = "Dark-300s-*.fit"
FLAT_FILES = "flat-*.fit"
CALIB_FILES = "neon-15s-*.fit"
SCIENCE_FILES = "deneb-200s-*.fit"
TRIM_REGION = None   
EXPOSURE_KEY = 'EXPTIME'

In [ ]:
CAPTURE_DIR='../../../CAPTURES/20231007_Void/'
BIAS_FILES = "Bias-*.fit"
DARK_FILES = "Dark-300s-*.fit"
FLAT_FILES = "flat-*.fit"
CALIB_FILES = "neon-10s-*.fit"
SCIENCE_FILES = "10lac-300s-*.fit"
TRIM_REGION = None
TRIM_REGION = '0, 600, 1600, 700'  
EXPOSURE_KEY = 'EXPTIME'

In [ ]:
CAPTURE_DIR='../../../CAPTURES/HAT-P-32b/'
BIAS_FILES = "d*BIASR*.fit"
DARK_FILES = None
FLAT_FILES = "d*FLATR*.fit"
CALIB_FILES = None
SCIENCE_FILES = "d*UW32R10*.fit"
TRIM_REGION = None
EXPOSURE_KEY = 'EXPOSURE'

In [ ]:
import fnmatch, os
from astropy import units as u
from astropy.nddata import CCDData
from ccdproc import ImageFileCollection
from combiner import Images

### collect and sort (according to fit header 'date') file names using a wildcard filter
def find_files(directory: str, files_filter: str, sort_key: str = 'date'):
    ic = ImageFileCollection(directory, glob_include=files_filter)
    ic.sort([sort_key])
    return (ic.files_filtered(include_path=True))

### create master bias
if (BIAS_FILES is not None) and not (os.path.exists(CAPTURE_DIR + 'masterbias.fit')):
    master_bias = Images.from_fit(find_files(CAPTURE_DIR, BIAS_FILES)) \
                            .trim(TRIM_REGION) \
                            .median() 
    master_bias.write(CAPTURE_DIR + 'masterbias.fit', overwrite=True) 

### create master dark
if (DARK_FILES is not None) and not (os.path.exists(CAPTURE_DIR + 'masterdark.fit')):
    master_dark = Images.from_fit(find_files(CAPTURE_DIR, DARK_FILES)) \
                                .trim(TRIM_REGION) \
                                .bias_substract(master_bias) \
                                .median()
    master_dark.write(CAPTURE_DIR + 'masterdark.fit', overwrite=True) 

### create master flat
if (FLAT_FILES is not None) and not (os.path.exists(CAPTURE_DIR + 'masterflat.fit')):
    master_flat = Images.from_fit(find_files(CAPTURE_DIR, FLAT_FILES)) \
                                .trim(TRIM_REGION) \
                                .bias_substract(master_bias) \
                                .median()
    master_flat.write(CAPTURE_DIR + 'masterflat.fit', overwrite=True) 

### reduce and align science frames
master_sciences = Images.from_fit(find_files(CAPTURE_DIR, SCIENCE_FILES)) \
                            .trim(TRIM_REGION) \
                            .bias_substract(master_bias) \
                            .dark_substract(master_dark, scale_exposure = True, exposure = EXPOSURE_KEY) \
                            .flat_divide(master_flat) \
                            .spec_align(ref_image_index = 0) 
#                            .star_align(ref_image_index = 0)
for i, frame in zip(range(0, len(master_sciences)), master_sciences):
    frame.write(f'{CAPTURE_DIR}science_reduced_{i}.fit', overwrite=True)

### combine science frames
master_science = master_sciences.sum() 
#master_science = master_sciences.median()
master_science.write(CAPTURE_DIR + 'masterscience.fit', overwrite=True) 


In [ ]:
plt.close('all')
#master_bias = CCDData.read(CAPTURE_DIR + 'masterbias.fit')

#show_image(master_bias)
#show_image(master_dark)
#show_image(master_flat)
#show_image(master_sciences[5])
#show_image(master_sciences[6])

show_image(master_science)

In [ ]:
CAPTURE_DIR='../../../CAPTURES/NY_Vir/'
BIAS_FILES = "offset-15c-Bin2-*.fits"
DARK_FILES = "darkBin2X-35C-30s-*.fits"
FLAT_FILES = "Flat_v-*.fits"
CALIB_FILES = ""
SCIENCE_FILES = "NY VIR2*.fits"
TRIM_REGION = None
EXPOSURE_KEY = 'EXPOSURE'

In [ ]:
import warnings
import numpy as np
from astropy.nddata import CCDData
from astropy.utils.exceptions import AstropyWarning
from ccdproc import ImageFileCollection, Combiner, combine
warnings.simplefilter('ignore', category=AstropyWarning)
warnings.simplefilter('ignore', UserWarning)


In [ ]:
ic = ImageFileCollection(CAPTURE_DIR, glob_include=SCIENCE_FILES)
ic.sort(['date'])
ic.files_filtered(include_path=True)
#ic.summary.colnames
#ic.sort(['date'])
#print(ic)


In [ ]:
#ic = ImageFileCollection(CAPTURE_DIR, glob_include=BIAS_FILES)
#print(ic.files_filtered(include_path=True))
master_bias = combine(img_list = ic.files_filtered(include_path=True),
                    method = 'median',  
                    dtype = np.float32, 
                    unit = 'u.adu', 
                    output_file = CAPTURE_DIR + 'masterbias_3.fit',
                    overwrite_output = True,
                    mem_limit=1e9)


In [ ]:
master_bias.dtype
master_bias.write(CAPTURE_DIR + 'masterbias_2.fit', overwrite=True) 

In [ ]:
for hdu in ic.hdus(save_with_name='_new'):
   hdu.header['meansub'] = True
   hdu.data = hdu.data - hdu.data.mean()

In [ ]:
hdu.header